In [19]:
%run ../setup_db.ipynb
conf = SandboxConfig(EXTERNAL_HOST_NAME="127.0.0.1", HOST_PORT=8563, BUCKETFS_PORT=2580)
setup_schema(conf)

Schema created in 55.96ms


Notebook setup [cloud-storage-extension](https://github.com/exasol/cloud-storage-extension) in the DB

- [x] retrieve the last released jar from github 
- [ ] put it in bucketfs of our DB
- [ ] setup connectors

In [2]:
import requests
import pathlib
import typing as tt

# Retrieve the latest released jar file

In [3]:
def get_latest_version_and_jar_url() -> tt.Tuple[str, str]:
    r = requests.get("https://api.github.com/repos/exasol/cloud-storage-extension/releases/latest")
    if r.status_code != 200:
        raise RuntimeError("Error sending request to the github api, code: %d" % r.status_code)
    data = r.json()
    version = data.get('tag_name')
    if version is None:
        raise RuntimeError("The latests version has no tag, something is wrong")
    for asset in data.get('assets', []):
        name = asset['name']
        if name.endswith(f"{version}.jar"):
            dl_url = asset['browser_download_url']
            return version, dl_url
    raise RuntimeError("Could not find proper jar url for the latest release")

In [10]:
version, jar_url = get_latest_version_and_jar_url()
_, local_jar_name = jar_url.rsplit('/', maxsplit=1)
local_jar_path = pathlib.Path(local_jar_name)
if local_jar_path.exists():
    print(f"Jar for version {version} already exists in {local_jar_path}, skip downloading")
else:
    print(f"Fetching jar for version {version} from {jar_url}...")
    r = requests.get(jar_url, stream=True)
    count_bytes = local_jar_path.write_bytes(r.content)
    print(f"Saved {count_bytes} bytes in {local_jar_path}")

Fetching jar for version 2.7.6 from https://github.com/exasol/cloud-storage-extension/releases/download/2.7.6/exasol-cloud-storage-extension-2.7.6.jar...
Saved 572649732 bytes in exasol-cloud-storage-extension-2.7.6.jar


# Upload the jar to the bucketfs

In [38]:
from exasol import bucketfs

In [46]:
UDFS_BUCKET = "myudfs"

In [39]:
bfs_url = conf.BUCKETFS_URL_PREFIX + conf.EXTERNAL_BUCKETFS_HOST
bfs_creds = {
    UDFS_BUCKET: {
        "username": conf.BUCKETFS_USER,
        "password": conf.BUCKETFS_PASSWORD,
    } 
}

In [40]:
svc = bucketfs.Service(bfs_url, bfs_creds)

In [42]:
bucket = svc[UDFS_BUCKET]

In [50]:
jar_exists = local_jar_name in list(bucket)
if jar_exists:
    print("Jar file is already present in the bucketfs")
else:
    print("Upload jar to bucketfs")
    with local_jar_path.open("rb") as fd:
        bucket.upload(local_jar_name, fd)    

Jar file is already present in the bucketfs


# Create connection script